In [9]:
from bs4 import BeautifulSoup
import pandas as pd

f=open('new.html',encoding='utf-8')
# Extract Section 1: User Information
user_info_table = soup.find('table', id='user-info')
user_data = []
for row in user_info_table.find_all('tr'):
    columns = row.find_all('td')
    if columns:
        user_data.append([col.text for col in columns])
user_df = pd.DataFrame(user_data, columns=['Name', 'Age', 'Email'])
print("User Information DataFrame:")
print(user_df)

# Extract Section 2: Product Information
product_info_table = soup.find('table', id='product-info')
product_data = []
for row in product_info_table.find_all('tr'):
    columns = row.find_all('td')
    if columns:
        product_data.append([col.text for col in columns])
product_df = pd.DataFrame(product_data, columns=['Product', 'Price', 'Stock'])
print("\nProduct Information DataFrame:")
print(product_df)

# Extract Section 3: Nested Information
nested_info_div = soup.find('div', id='nested-info')
category_elements = nested_info_div.find_all(['h3', 'li'])

current_category = None
nested_data = []

for element in category_elements:
    if element.name == 'h3':
        current_category = element.text.strip()
    elif element.name == 'li':
        item_name = element.text.strip().split('\n')[0]
        nested_ul = element.find('ul')
        if nested_ul:
            details = nested_ul.find_all('li')
            for detail in details:
                nested_data.append([current_category, item_name, detail.text.strip()])
        else:
            nested_data.append([current_category, item_name, None])

# Convert to DataFrame
nested_df = pd.DataFrame(nested_data, columns=['Category', 'Item', 'Detail'])

# Filter out rows with None in 'Detail' column only if 'Item' is an actual item without nested details
valid_data = []
for row in nested_df.values:
    if row[2] is not None:  # Detail is not None
        valid_data.append(row)
    else:
        # Check if the Item should indeed have None for Detail
        if 'Detail' not in row[1]:
            valid_data.append(row)

nested_df = pd.DataFrame(valid_data, columns=['Category', 'Item', 'Detail'])

print("\nNested Information DataFrame:")
print(nested_df)

User Information DataFrame:
    Name Age              Email
0  Alice  30  alice@example.com
1    Bob  25    bob@example.com

Product Information DataFrame:
     Product  Price Stock
0  Product A  10.99   100
1  Product B  20.99   200

Nested Information DataFrame:
     Category      Item        Detail
0  Category 1  Item 1.1  Detail 1.1.1
1  Category 1  Item 1.1  Detail 1.1.2
2  Category 1  Item 1.2          None
3  Category 2  Item 2.1  Detail 2.1.1
4  Category 2  Item 2.2          None
